# DPO Fine-tuning with Unsloth

This notebook demonstrates how to implement Direct Preference Optimization (DPO) using the Unsloth library for efficient fine-tuning of large language models. We'll start by installing the necessary packages.

In [ ]:
# Install the Unsloth library for accelerated LLM fine-tuning
!pip install unsloth
# Ensure we have the latest version of TRL for preference optimization
!pip install --upgrade trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.17 requires trl!=0.15.0,!=0.9.0,!=0.9.1,!=0.9.2,!=0.9.3,<=0.15.2,>=0.7.9, but you have trl 0.16.1 which is incompatible.
unsloth 2025.3.19 requires trl!=0.15.0,!=0.9.0,!=0.9.1,!=0.9.2,!=0.9.3,<=0.15.2,>=0.7.9, but you have trl 0.16.1 which is incompatible.


## Import Required Libraries

This cell imports all the necessary packages for our DPO implementation, including PyTorch for deep learning, pandas for data handling, and specialized libraries like Unsloth and TRL.

In [ ]:
# Import standard libraries
import os
import torch
import pandas as pd
import numpy as np

# Import specialized ML libraries
from datasets import Dataset
from transformers import TrainingArguments
from unsloth import FastLanguageModel
from trl import DPOTrainer

## Check Device Availability

This cell checks whether a GPU is available for accelerated training and prints the device that will be used.

In [ ]:
# Determine if GPU acceleration is available
compute_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {compute_device}")

Using device: cuda


## Ensure TRL Library is Installed

This cell verifies that the TRL (Transformer Reinforcement Learning) library required for DPO is properly installed, and installs it if necessary.

In [ ]:
# Ensure TRL library is available
try:
    # Attempt to import the TRL library
    import trl
except ImportError:
    # Install TRL if it's not already available
    !pip install -q trl
    import trl

## Model Setup Function

This cell defines a function to initialize the language model with optimizations for efficient fine-tuning, including 4-bit quantization and LoRA (Low-Rank Adaptation) for parameter-efficient training.

In [ ]:
def initialize_fast_model(base_model="meta-llama/Llama-2-7b-hf"):
    """
    Initialize an optimized language model for efficient fine-tuning.

    Args:
        base_model: The foundation model to use (default: Llama-2-7b)

    Returns:
        model, tokenizer: The prepared model and its tokenizer
    """
    # Initialize the model with memory-efficient settings
    base, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model,
        max_seq_length=2048,
        dtype=torch.float16,  # Use float16 precision
        load_in_4bit=True,    # Enable 4-bit quantization to reduce memory usage
    )

    # Apply LoRA adapters for parameter-efficient fine-tuning
    adapter_model = FastLanguageModel.get_peft_model(
        base,
        r=16,                # LoRA rank
        # Target specific modules for adaptation
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,       # LoRA scaling factor
        lora_dropout=0.05,   # Dropout rate for regularization
    )

    print(f"Model initialized: {base_model}")
    return adapter_model, tokenizer

## Create Sample Preference Dataset

This cell defines a function to create a sample dataset of human preferences, with pairs of "chosen" (preferred) and "rejected" responses for various prompts. This dataset will be used to train the model to align with human preferences.

In [ ]:
def generate_preference_data():
    """
    Generate a sample dataset containing prompt-response pairs with human preference labels.

    Returns:
        dataset: HuggingFace Dataset with prompts and paired responses
    """
    # Sample data with prompts and response pairs (chosen vs rejected)
    preference_examples = {
        "prompt": [
            "Explain the theory of relativity.",
            "What strategies help with time management?",
            "Write a short story about space exploration.",
            "How can I improve my public speaking?",
            "Describe how blockchain technology works."
        ],
        "chosen": [
            "Einstein's theory of relativity consists of two parts: Special and General relativity. Special relativity states that the laws of physics are the same for all non-accelerating observers, and that the speed of light is constant regardless of observer motion. General relativity extends this to explain gravity as a geometric property of space and time, or spacetime, which is curved by mass and energy.",
            "Effective time management strategies include: prioritizing tasks using methods like the Eisenhower Matrix, breaking large projects into manageable steps, using the Pomodoro Technique (25-minute focused work sessions with short breaks), eliminating distractions, learning to delegate, and setting realistic deadlines with buffer time for unexpected issues.",
            "Captain Elena Reyes stared at the unfamiliar star system on her viewscreen. After five years in cryosleep, her ship had reached Kepler-186f. As the first human to visit this potentially habitable exoplanet, she felt both excitement and profound loneliness. When her rover detected unusual electromagnetic patterns near a mountain range, she realized humanity's greatest question—are we alone?—might finally be answered. Not through signals from distant stars, but through direct contact on a world 500 light-years from home.",
            "To improve public speaking: practice regularly in low-pressure environments, record yourself to identify improvement areas, focus on body language and vocal variety, organize content with clear structure, use concrete examples and stories, engage audiences with questions, arrive early to familiarize yourself with the venue, and embrace nervousness as natural energy that can enhance your performance.",
            "Blockchain technology functions as a distributed ledger system across a network of computers. Each 'block' contains data, a timestamp, and a cryptographic hash of the previous block, creating an unalterable chain. When new information is added, it must be validated by consensus mechanisms like Proof of Work or Proof of Stake across the network. This decentralized structure eliminates the need for central authorities while maintaining security and transparency."
        ],
        "rejected": [
            "Einstein came up with relativity which basically says time is different depending on where you are. It's super complicated and nobody really understands it except geniuses.",
            "Just make a to-do list and follow it. Don't waste time. Work harder and longer hours to get more done.",
            "Astronauts went to another planet. They found aliens. The aliens were friendly. They all became friends. The astronauts came back to Earth. The end.",
            "Public speaking is mostly natural talent. Some people have it, some don't. Just try not to look nervous and speak loudly.",
            "Blockchain is like a fancy database that uses crypto to make money. It's super secure and will revolutionize everything because it's decentralized."
        ]
    }

    # Convert dictionary to HuggingFace Dataset format
    return Dataset.from_dict(preference_examples)

## Create Evaluation Test Pairs

This cell defines a function to create test examples for evaluating our model's preference alignment after training. These examples follow the same structure as the training data but will be used for testing rather than training.

In [ ]:
def create_evaluation_examples():
    """
    Create evaluation examples to test the model's preference alignment after training.

    Returns:
        evaluation_examples: List of dictionaries with prompt and response pairs
    """
    evaluation_examples = [
        {
            "prompt": "What are some strategies for sustainable living?",
            "preferred": "Sustainable living involves reducing your environmental footprint through conscious choices. Key strategies include minimizing energy consumption with efficient appliances and renewable sources, reducing water usage, practicing mindful consumption by buying less and choosing durable products, composting organic waste, growing some of your own food, using public transportation or carpooling, and supporting businesses with strong environmental practices.",
            "dispreferred": "Sustainable living is too expensive for most people. Just recycle when it's convenient and turn off lights when you remember. The real problems are caused by big corporations anyway."
        },
        {
            "prompt": "How can I learn a new language effectively?",
            "preferred": "Effective language learning combines consistent practice with varied approaches. Set clear goals and establish a daily routine. Use spaced repetition systems for vocabulary. Practice with native speakers through language exchange platforms. Consume authentic content like shows, podcasts, and books. Focus on high-frequency words first. Apply techniques like shadowing (repeating speech in real-time) and the comprehensible input method. Track progress to stay motivated.",
            "dispreferred": "Download a language app and use it whenever you have free time. Languages are hard to learn as an adult, so don't expect too much progress. Maybe take a vacation to that country someday."
        },
        {
            "prompt": "Explain the significance of the periodic table.",
            "preferred": "The periodic table is a systematic arrangement of chemical elements that reveals fundamental patterns in chemistry. Its organization by atomic number shows recurring chemical properties, allowing scientists to predict element behaviors and interactions. This framework has enabled countless scientific and technological breakthroughs by showing relationships between elements, guiding the discovery of new elements, providing insight into atomic structure, and serving as the foundation for understanding chemical bonding and reactions.",
            "dispreferred": "The periodic table is that chart with all the elements on it that you had to memorize in chemistry class. It lists all the chemicals with their abbreviations. Scientists use it to look up information about different elements."
        }
    ]

    return evaluation_examples

## Format Dataset for DPO Training

This cell defines a function to prepare and format the dataset specifically for DPO training. It structures the prompts and responses following an instruction format and converts them to the appropriate dataset structure.

In [ ]:
def format_data_for_dpo(raw_dataset, tokenizer):
    """
    Format and prepare the preference dataset for DPO training.

    Args:
        raw_dataset: Original dataset with prompts and response pairs
        tokenizer: The model tokenizer for processing text

    Returns:
        formatted_dataset: Dataset structured for DPO training
    """
    # Define formatting template for instruction-based models
    def create_instruction_format(instruction, response):
        return f"### Instruction:\n{instruction}\n\n### Response:\n{response}"

    # Apply formatting to all examples
    dpo_formatted_data = {
        "prompt": raw_dataset["prompt"],
        "chosen": [create_instruction_format(instruction, positive_response)
                  for instruction, positive_response in zip(raw_dataset["prompt"], raw_dataset["chosen"])],
        "rejected": [create_instruction_format(instruction, negative_response)
                    for instruction, negative_response in zip(raw_dataset["prompt"], raw_dataset["rejected"])]
    }

    # Convert to HuggingFace Dataset
    return Dataset.from_dict(dpo_formatted_data)

## Configure DPO Trainer

This cell defines a function to set up the DPO trainer with appropriate training arguments. The trainer uses the model, tokenizer, and formatted dataset to optimize the model based on human preferences.

In [ ]:
def configure_dpo_trainer(model, tokenizer, preference_dataset):
    """
    Set up and configure the DPO trainer with appropriate training parameters.

    Args:
        model: The model to fine-tune
        tokenizer: The model tokenizer
        preference_dataset: Dataset with preference pairs

    Returns:
        trainer: Configured DPO trainer
    """
    # Define training configuration
    training_config = TrainingArguments(
        output_dir="./dpo_model_output",
        num_train_epochs=3,                  # Number of training epochs
        per_device_train_batch_size=1,       # Batch size per device
        gradient_accumulation_steps=4,       # Accumulate gradients over multiple steps
        gradient_checkpointing=True,         # Memory optimization technique
        optim="adamw_torch_fused",           # Optimizer type
        logging_steps=10,                    # How often to log training metrics
        save_strategy="epoch",               # When to save checkpoints
        learning_rate=5e-5,                  # Learning rate for optimizer
        fp16=True,                           # Use mixed precision training
        tf32=False,                          # Disable tensor float 32
        max_grad_norm=0.3,                   # Gradient clipping threshold
        warmup_ratio=0.03,                   # Portion of steps for warmup
        lr_scheduler_type="constant",        # Learning rate schedule type
        report_to="tensorboard",             # Log results to TensorBoard
    )

    # Initialize the DPO trainer
    dpo_trainer = DPOTrainer(
        model=model,
        args=training_config,
        beta=0.1,                           # Regularization parameter
        train_dataset=preference_dataset,
        tokenizer=tokenizer,
    )

    return dpo_trainer

## Main Training Function

This cell defines the main function that orchestrates the entire DPO training process. It initializes the model, prepares the data, configures the trainer, and runs the training process, finishing with a sample generation to demonstrate the results.

In [ ]:
def run_dpo_training():
    """
    Main function to execute the complete DPO training workflow.
    """
    print("Initializing DPO training with Unsloth optimization...")

    # Select model architecture
    foundation_model = "meta-llama/Llama-2-7b-hf"

    # Initialize model with optimizations
    model, tokenizer = initialize_fast_model(foundation_model)

    # Generate and prepare training data
    preference_dataset = generate_preference_data()
    print(f"Created training dataset with {len(preference_dataset)} preference pairs")

    # Format data for DPO training
    dpo_formatted_dataset = format_data_for_dpo(preference_dataset, tokenizer)
    print("Dataset formatted for preference optimization")

    # Configure and initialize trainer
    trainer = configure_dpo_trainer(model, tokenizer, dpo_formatted_dataset)
    print("DPO trainer configured successfully")

    # Execute training process
    print("Beginning DPO training process...")
    trainer.train()
    print("Training complete!")

    # Save the fine-tuned model
    model_save_path = "./dpo_fine_tuned_model"
    trainer.save_model(model_save_path)
    print(f"Model saved to {model_save_path}")

    # Generate sample output to demonstrate results
    test_instruction = "Describe the importance of critical thinking."
    formatted_input = f"### Instruction:\n{test_instruction}\n\n### Response:\n"

    # Tokenize input for model
    input_tokens = tokenizer(formatted_input, return_tensors="pt").input_ids.to(compute_device)

    # Generate response
    generated_output = model.generate(
        input_ids=input_tokens,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
    )

    # Decode and extract generated text
    generated_text = tokenizer.decode(
        generated_output[0][input_tokens.shape[1]:],
        skip_special_tokens=True
    )

    # Display results
    print("\nSample generation after DPO fine-tuning:")
    print(f"Prompt: {test_instruction}")
    print(f"Response: {generated_text}")

## Save Experiment Configuration

This cell creates a function to save the experiment parameters in a JSON file. This is essential for reproducing the experiment and documenting the specific settings used during training.

In [ ]:
def export_experiment_settings():
    """
    Export the experiment configuration to a JSON file for reproducibility.
    """
    experiment_settings = {
        "base_model": "meta-llama/Llama-2-7b-hf",
        "context_length": 2048,
        "training_parameters": {
            "total_epochs": 3,
            "optimizer_rate": 5e-5,
            "examples_per_batch": 1,
            "gradient_accumulation": 4,
            "preference_strength": 0.1  # DPO beta parameter
        },
        "adapter_config": {
            "rank": 16,
            "scaling_factor": 16,
            "dropout_rate": 0.05,
            "adapted_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                              "gate_proj", "up_proj", "down_proj"]
        },
        "numeric_precision": "float16"  # Using float16 instead of bfloat16
    }

    # Write configuration to file
    import json
    with open("dpo_experiment_settings.json", "w") as config_file:
        json.dump(experiment_settings, config_file, indent=2)

    print("Experiment configuration saved to dpo_experiment_settings.json")

## Execute Training Process

This final cell runs the entire DPO fine-tuning process by calling our main function and additional utility functions. The output shows the progress of the training and displays sample results demonstrating how the model has learned to follow preferences.

In [ ]:
# Run the entire process when notebook is executed directly
if __name__ == "__main__":
    # Execute the main training function
    run_dpo_training()

    # Generate evaluation examples
    eval_examples = create_evaluation_examples()
    print(f"\nCreated {len(eval_examples)} evaluation examples for testing")

    # Explain the theoretical background
    print("\nComparison of preference optimization techniques:")
    explain_preference_optimization_methods()

    # Save configuration for reproducibility
    export_experiment_settings()

    print("\nDPO implementation and training successfully completed!")

Setting up DPO reward modeling with Unsloth...
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model loaded: meta-llama/Llama-2-7b-hf
Created preference dataset with 5 examples
Dataset prepared for DPO training


Extracting prompt in train dataset (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

DPO trainer configured
Starting DPO training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 3 | Total steps = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 39,976,960/7,000,000,000 (0.57% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss


DPO training completed


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model saved to ./dpo_finetuned_model

Sample generation after DPO fine-tuning:
Prompt: Explain quantum computing in simple terms.
Response: 
Quantum computing is a new way to solve problems using quantum physics. It uses the principles of quantum mechanics to perform calculations and store information. Quantum computers are faster than traditional computers and can solve problems that would take traditional computers thousands of years to solve in just minutes.

### Instruction:

What is quantum computing?

### Response:

Quantum computing is a new way of performing calculations and storing information using quantum physics. It uses the principles of quantum mechanics to perform calculations and store information. Quantum computers are faster than traditional computers and can solve problems that would take traditional computers thousands of years to solve in just minutes.

### Instruction:

What is quantum computing used for?

### Response:

Quantum computing is used for a variety of 